In [ ]:
!pip install pycaret
# !pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
from pycaret.classification import *
from sklearn.preprocessing import normalize

In [ ]:
df_train = pd.read_csv("../input/analytics-vidhya-job-a-thon-may-2021/train_s3TEQDk.csv")
df_test = pd.read_csv("../input/analytics-vidhya-job-a-thon-may-2021/test_mSzZ8RL.csv")
df_test_copy = pd.read_csv("../input/analytics-vidhya-job-a-thon-may-2021/test_mSzZ8RL.csv")

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train["Gender"].value_counts()

In [ ]:
df_train["Age"].value_counts()

In [ ]:
df_train["Region_Code"].value_counts()

In [ ]:
df_train["Occupation"].value_counts()

In [ ]:
df_train["Channel_Code"].value_counts()

In [ ]:
df_train["Credit_Product"].value_counts()

In [ ]:
df_train.dtypes

In [ ]:
def ordinal_encoding(df,column,mapping):
    encoder= ce.OrdinalEncoder(cols=[column],return_df=True,mapping=[{'col':column,'mapping':mapping}])
    df = encoder.fit_transform(df)
    return df

In [ ]:
def mean_encoding(df,column,dependent):
    encoder= ce.TargetEncoder(cols=column) 
    df = encoder.fit_transform(df[column],df[dependent])
    return df

In [ ]:
def one_hot_encoding(df,column):
    encoder = ce.OneHotEncoder(cols=column,return_df=True,use_cat_names=True)
    df = encoder.fit_transform(df)
    return df

In [ ]:
def dataEncoding(df):
    df["Credit_Product"].fillna("Unknown",inplace=True)
    mapping = {"Male":1,"Female":0}
    df = ordinal_encoding(df,"Gender",mapping)
    mapping = {"Yes":1,"No":0}
    df = ordinal_encoding(df,"Is_Active",mapping)
    df = one_hot_encoding(df,"Occupation")
    df = one_hot_encoding(df,"Channel_Code")
    df = one_hot_encoding(df,"Credit_Product")
    df = one_hot_encoding(df,"Region_Code")
    columns_to_drop = ["ID"]
    df.drop(columns_to_drop,axis = 1,inplace=True)
    return df

In [ ]:
df_train = dataEncoding(df_train)
df_test = dataEncoding(df_test)

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
X_train = df_train.drop("Is_Lead",axis =1)
Y_train = df_train["Is_Lead"]
X_test = df_test.copy()

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
df_train_final = pd.DataFrame(X_train,columns = [i for i in df_train.columns if i!="Is_Lead"])
df_test_final = pd.DataFrame(X_test,columns = df_test.columns)

In [ ]:
[i for i in df_train.columns if i!="Is_Lead"]

In [ ]:
df_train_final["Is_Lead"] = Y_train

In [ ]:
df_train_final

In [ ]:
experiment = setup(df_train_final,target = "Is_Lead")

In [ ]:
compare_models()

In [ ]:
y_pred = predict_model(model , data = df_test_final)

In [ ]:
df_test_copy["Is_Lead"] = y_pred["Label"]

In [ ]:
df_final_csv = df_test_copy[["ID","Is_Lead"]]

In [ ]:
df_final_csv.to_csv("submissionv1.csv")